Setup Ambiente: Clone Repo & Recupero Dati

In [1]:
import os
import shutil
import sys
from pathlib import Path
from google.colab import drive

# --- CONFIGURAZIONE ---
GITHUB_REPO = "https://github.com/micheleapu01/Artificial_Vision_Contest.git"
REPO_NAME = GITHUB_REPO.split("/")[-1].replace(".git", "")
DRIVE_FOLDER_NAME = "Gr12 - AVContestDelivery"

# ==============================================================================
# A. MONTAGGIO DRIVE
# ==============================================================================
if not os.path.exists('/content/drive'):
    drive.mount('/content/drive')

BASE_DRIVE_PATH = Path(f"/content/drive/MyDrive/{DRIVE_FOLDER_NAME}")

if not BASE_DRIVE_PATH.exists():
    found = list(Path("/content/drive/MyDrive").glob(f"*{DRIVE_FOLDER_NAME}*"))
    if found:
        BASE_DRIVE_PATH = found[0]
    else:
        print(f" Cartella Drive '{DRIVE_FOLDER_NAME}' non trovata.")
        raise FileNotFoundError("Cartella dati mancante.")

print(f" Fonte Dati (Drive): {BASE_DRIVE_PATH}")

# ==============================================================================
# B. CLONE REPOSITORY
# ==============================================================================
WORK_DIR = Path(f"/content/{REPO_NAME}")

if WORK_DIR.exists():
    shutil.rmtree(WORK_DIR)

print(f" Clonazione Repository GitHub...")
!git clone $GITHUB_REPO > /dev/null 2>&1
print(" Repository clonata.")

# ==============================================================================
# C. PREPARAZIONE CARTELLE E PESI
# ==============================================================================
print("\n Preparazione ambiente...")
# 1. Copia i PESI (Da Drive -> Locale)
SRC_WEIGHTS = BASE_DRIVE_PATH / "weights"
DST_WEIGHTS = WORK_DIR / "weights"

if SRC_WEIGHTS.exists():
    if DST_WEIGHTS.exists(): shutil.rmtree(DST_WEIGHTS)
    shutil.copytree(SRC_WEIGHTS, DST_WEIGHTS)
    print("    Pesi copiati (YOLO + ReID).")
else:
    print(f"    ERRORE: Non trovo la cartella 'weights' in {BASE_DRIVE_PATH}")

# 2. Creazione Struttura Video
TARGET_VIDEO_DIR = WORK_DIR / "SIMULATOR/lecture_example_from_training/test_set/videos"
TARGET_VIDEO_DIR.mkdir(parents=True, exist_ok=True)

print(f"    Cartella video creata: .../{TARGET_VIDEO_DIR.name}")

# ==============================================================================
# D. INSTALLAZIONE DIPENDENZE
# ==============================================================================
os.chdir(WORK_DIR)
if os.path.exists("requirements.txt"):
    print("\n Installazione librerie...")
    !pip install -r requirements.txt -q

print(f"\n AMBIENTE PRONTO IN: {os.getcwd()}")

Mounted at /content/drive
 Fonte Dati (Drive): /content/drive/MyDrive/Gr12 - AVContestDelivery
 Clonazione Repository GitHub...
 Repository clonata.

 Preparazione ambiente...
    Pesi copiati (YOLO + ReID).
    Cartella video creata: .../videos

 Installazione librerie...
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.8/46.8 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 69.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 72.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.9/57.9 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 138.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 256.2/256.2 kB 24.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.6/63.6 kB 6.8 MB/s eta 0:00:00

 AMBIENTE PRONTO IN: /content/Artificial_Vision_Contest


Caricamento video di test

In [2]:
from tqdm.notebook import tqdm

# ==============================================================================
# CONFIGURAZIONE PERCORSI
# ==============================================================================
# Percorso SORGENTE su Drive in cui sono contenuti i video su cui si vuole testare
DRIVE_SOURCE_PATH = "/content/drive/MyDrive/Colab Notebooks/Artificial Vision/Contest/test_set/videos"

# Percorso DESTINAZIONE Locale
DEST_DIR = Path("SIMULATOR/lecture_example_from_training/test_set/videos")

print(f" Sorgente Drive:      {DRIVE_SOURCE_PATH}")
print(f" Destinazione Locale: {DEST_DIR}")
print("-" * 60)

# ==============================================================================
# FUNZIONI DI UTILITÀ
# ==============================================================================
def count_files(directory):
    """Conta tutti i file nelle sottocartelle per la progress bar"""
    count = 0
    for root, dirs, files in os.walk(directory):
        count += len(files)
    return count

def copy_with_progress(src, dst):
    """Copia ricorsiva con barra di caricamento"""
    src = Path(src)
    dst = Path(dst)

    if not src.exists():
        print(f" ERRORE: Percorso sorgente non trovato: {src}")
        return

    # 1. Contiamo i file totali
    total_files = count_files(src)
    print(f"   -> Trovati {total_files} file.")

    # 2. Copia con progress bar
    with tqdm(total=total_files, desc="Copia Dataset", unit="file") as pbar:
        for root, dirs, files in os.walk(src):
            # Calcoliamo il percorso relativo (es. "3/img1")
            rel_path = os.path.relpath(root, src)
            dest_folder = dst / rel_path

            # Creiamo la cartella di destinazione
            dest_folder.mkdir(parents=True, exist_ok=True)

            # Copiamo i file uno ad uno
            for file in files:
                src_file = Path(root) / file
                dst_file = dest_folder / file

                shutil.copy2(src_file, dst_file)
                pbar.update(1)

# ==============================================================================
# ESECUZIONE
# ==============================================================================
if DEST_DIR.exists():
    shutil.rmtree(DEST_DIR)
DEST_DIR.mkdir(parents=True, exist_ok=True)

# Avvio Copia
copy_with_progress(DRIVE_SOURCE_PATH, DEST_DIR)

print("\n Importazione Completata!")

# Verifica veloce
sequences = [f.name for f in DEST_DIR.iterdir() if f.is_dir()]
print(f"   Sequenze video importate ({len(sequences)}): {sorted(sequences)}")

 Sorgente Drive:      /content/drive/MyDrive/Colab Notebooks/Artificial Vision/Contest/test_set/videos
 Destinazione Locale: SIMULATOR/lecture_example_from_training/test_set/videos
------------------------------------------------------------
   -> Trovati 6048 file.


Copia Dataset:   0%|          | 0/6048 [00:00<?, ?file/s]


 Importazione Completata!
   Sequenze video importate (8): ['12', '20', '27', '3', '37', '4', '48', '6']


Esecuzione Pipeline & Salvataggio Risultati su Drive

In [3]:
import glob
import subprocess
import sys
from datetime import datetime


# ==============================================================================
# 0. SETUP PATH & SELEZIONE PESI
# ==============================================================================
repo_candidates = [p for p in Path("/content").iterdir() if p.is_dir() and (p / ".git").exists()]
WORK_DIR = repo_candidates[0] if repo_candidates else Path("/content/Artificial_Vision_Contest")
os.chdir(WORK_DIR)
print(f" Directory di Lavoro: {WORK_DIR}")

WEIGHTS_DIR = Path("weights")
print("\n Configurazione Modelli...")

# A. DETECTOR (Target: train_yolo11m_SoccerNet.pt)
target_det = WEIGHTS_DIR / "train_yolo11m_SoccerNet.pt"
if target_det.exists():
    DETECTOR_PATH = target_det
else:
    print(f"'{target_det.name}' non trovato. Cerco alternative...")
    try:
        DETECTOR_PATH = list(WEIGHTS_DIR.glob("*.pt"))[0]
    except IndexError:
        raise FileNotFoundError(" Nessun peso .pt trovato in weights!")

# B. ReID (Target: osnet_x0_25_finetuned_backbone_only.pth)
target_reid = WEIGHTS_DIR / "osnet_x0_25_finetuned_backbone_only.pth"
if target_reid.exists():
    REID_PATH = target_reid
else:
    print(f" '{target_reid.name}' non trovato. Cerco alternative...")
    try:
        REID_PATH = list(WEIGHTS_DIR.glob("*.pth"))[0]
    except IndexError:
         raise FileNotFoundError(" Nessun peso .pth trovato in weights!")

print(f"    Detector: {DETECTOR_PATH.name}")
print(f"    ReID:     {REID_PATH.name}")

# ==============================================================================
# SETUP DA GARA
# ==============================================================================
SOURCE_VIDEOS = "SIMULATOR/lecture_example_from_training/test_set/videos"
FULL_OUTPUT_DIR = "SIMULATOR/lecture_example_from_training/Predictions_folder"

SCRIPT_TRACK        = "scripts/osnet_reid_track.py"
SCRIPT_FIELD_FILTER = "data_tools/field_filter.py"
SCRIPT_INTERP       = "scripts/interpolate.py"
SCRIPT_STITCH       = "scripts/stitching.py"
SCRIPT_BEHAVIOR     = "scripts/behavior.py"
TRACKER_CONFIG      = "configs/botsort_with_reid_OsNet.yaml"

# ==============================================================================
# SETUP PARAMETRI
# ==============================================================================

# 1. TRACKING
TRACKING_ARGS = [
    "--source", SOURCE_VIDEOS,
    "--tracker", TRACKER_CONFIG,
    "--out", FULL_OUTPUT_DIR,
    "--weights", str(DETECTOR_PATH),
    "--conf", "0.15",
    "--iou", "0.6",
    "--osnet-weights", str(REID_PATH),
    "--max-det", "300",
    "--reid-min-h", "60",
    "--reid-pad", "0.10",
    "--hm-fusion"
]

# 2. FIELD FILTER
FILTER_ARGS = [
    "--pred-folder", FULL_OUTPUT_DIR,
    "--videos-root", SOURCE_VIDEOS,
    "--mask-every", "5",
    "--line-tol", "2"
]

# 3. STITCHING
STITCH_ARGS = [
    "--max-gap", "30",
    "--dist-base", "1.5",
    "--dist-per-gap", "0.15",
    "--min-len", "6",
    "--allow-border"
]

# 4. INTERPOLAZIONE
INTERP_ARGS = ["--gap", "30", "--conf", "0.4", "--sigma", "1.5"]

# 5. BEHAVIOR
BEHAVIOR_ARGS = ["--source", SOURCE_VIDEOS, "--tracking-dir", FULL_OUTPUT_DIR, "--out", FULL_OUTPUT_DIR, "--window", "7"]

# ==============================================================================
# ESECUZIONE
# ==============================================================================
def run_command(script, args, name):
    print(f"\n{'='*60}\n {name}\nCMD: python {script} {' '.join(args)}\n{'='*60}")
    try:
        subprocess.run([sys.executable, "-u", script] + args, check=True)
        print(f" {name} OK.")
    except Exception as e:
        print(f" ERRORE: {e}")
        sys.exit(1)

def main():
    if not os.path.exists(FULL_OUTPUT_DIR): os.makedirs(FULL_OUTPUT_DIR)

    # 1. TRACKING (Salta se esiste)
    if not glob.glob(os.path.join(FULL_OUTPUT_DIR, "tracking_*.txt")):
        run_command(SCRIPT_TRACK, TRACKING_ARGS, "Tracking")
    else:
        print(" Tracking già presente. Salto lo step...")

    # 2. FILTER
    run_command(SCRIPT_FIELD_FILTER, FILTER_ARGS, "Field Filter")

    # 3. PIPELINE RECORD
    run_command(SCRIPT_STITCH, ["--folder", FULL_OUTPUT_DIR] + STITCH_ARGS, "1. Stitching ")
    run_command(SCRIPT_INTERP, ["--folder", FULL_OUTPUT_DIR] + INTERP_ARGS, "2. Interpolazione")
    run_command(SCRIPT_BEHAVIOR, BEHAVIOR_ARGS, "3. Behavior")

    # ==============================================================================
    # SALVATAGGIO AUTOMATICO SU DRIVE
    # ==============================================================================
    DRIVE_DELIVERY_NAME = "Gr12 - AVContestDelivery"
    DEST_DRIVE_PATH = Path(f"/content/drive/MyDrive/{DRIVE_DELIVERY_NAME}")

    if DEST_DRIVE_PATH.exists():
        print(f"\n Salvataggio risultati su Drive...")
        timestamp = datetime.now().strftime("%Y%m%d_%H%M")
        zip_path = DEST_DRIVE_PATH / f"Predictions_{timestamp}"
        shutil.make_archive(str(zip_path), 'zip', FULL_OUTPUT_DIR)
        print(f" Archivio salvato in: {zip_path}.zip")
    else:
        print(f"\n Risultati salvati solo in locale: {FULL_OUTPUT_DIR}")

if __name__ == "__main__":
    main()

 Directory di Lavoro: /content/Artificial_Vision_Contest

 Configurazione Modelli...
    Detector: train_yolo11m_SoccerNet.pt
    ReID:     osnet_x0_25_finetuned_backbone_only.pth

 Tracking
CMD: python scripts/osnet_reid_track.py --source SIMULATOR/lecture_example_from_training/test_set/videos --tracker configs/botsort_with_reid_OsNet.yaml --out SIMULATOR/lecture_example_from_training/Predictions_folder --weights weights/train_yolo11m_SoccerNet.pt --conf 0.15 --iou 0.6 --osnet-weights weights/osnet_x0_25_finetuned_backbone_only.pth --max-det 300 --reid-min-h 60 --reid-pad 0.10 --hm-fusion
 Tracking OK.

 Field Filter
CMD: python data_tools/field_filter.py --pred-folder SIMULATOR/lecture_example_from_training/Predictions_folder --videos-root SIMULATOR/lecture_example_from_training/test_set/videos --mask-every 5 --line-tol 2
 Field Filter OK.

 1. Stitching 
CMD: python scripts/stitching.py --folder SIMULATOR/lecture_example_from_training/Predictions_folder --max-gap 30 --dist-base 1.5 

# Conclusione e Istruzioni per la Visualizzazione
La pipeline è stata eseguita e i file di tracking (.txt) sono stati generati correttamente.

I risultati sono stati salvati automaticamente nel Google Drive montato, all'interno della cartella di consegna:

Percorso: Gr12 - AVContestDelivery/Predictions_YYYYMMDD_HHMM.zip

Contenuto: L'archivio contiene tutti i file .txt necessari per la valutazione.

Poiché il rendering grafico su Colab è limitato, si esegue la simulazione sulla propria macchina locale utilizzando i file prodotti:



1.   Scaricare lo zip Predictions_... dal Drive.
1.   Estrarre i file nella cartella Predictions_folder del SIMULATOR
2.   Lanciare lo script